```
This notebook sets up and runs a set of benchmarks to compare
different numerical discretizations of the SWEs

Copyright (C) 2016  SINTEF ICT

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Import packages we need
import numpy as np
from matplotlib import animation, rc
from matplotlib import pyplot as plt

import os
import pycuda.driver as cuda
from pycuda.compiler import SourceModule
import datetime
import sys

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

#Import our simulator
from SWESimulators import FBL, CTCS, LxF, KP07, CDKLM16, PlotHelper, Common, WindStress
#Import initial condition and bathymetry generating functions:
from SWESimulators.BathymetryAndICs import *

In [ ]:
#Create output directory for images
imgdir='images_' + datetime.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
os.makedirs(imgdir)
print "Saving images to " + imgdir

In [ ]:
#Create GPU Context
gpu_ctx = Common.CUDAContext()

In [ ]:
#Clean up old simulator if any:
if 'sim' in globals():
    sim.cleanUp()
    
# Forward backward linear
nx = 100
ny = 200

dx = 200.0
dy = 200.0

dt = 1
g = 9.81
f = 0.0
r = 0.0

#wind = WindStress.UniformAlongShoreWindStress(tau0=3.0, rho=1025, alpha=1.0/(100*dx)) 
wind =  WindStress.NoWindStress()


bcSettings = 1
if (bcSettings == 1):
    boundaryConditions = Common.BoundaryConditions()
    ghosts = [0,0,0,0] # north, east, south, west
elif (bcSettings == 2):
    boundaryConditions = Common.BoundaryConditions(2,2,2,2)
    ghosts = [1,1,0,0] # Both periodic
elif bcSettings == 3:
    boundaryConditions = Common.BoundaryConditions(2,1,2,1)
    ghosts = [1,0,0,0] # periodic north-south
else:
    boundaryConditions = Common.BoundaryConditions(1,2,1,2)
    ghosts = [0,1,0,0] # periodic east-west


dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])
    
h0 = np.ones(dataShape, dtype=np.float32, order='C') * 60;
#addTopographyBump(h0, nx, ny, dx, dy, ghosts, 40)

eta0 = np.zeros(dataShape, dtype=np.float32, order='C');
u0 = np.zeros((dataShape[0], dataShape[1]+1), dtype=np.float32, order='C');
v0 = np.zeros((dataShape[0]+1, dataShape[1]), dtype=np.float32, order='C'); 
    
#Create bump in to lower left of domain for testing
addCentralBump(eta0, nx, ny, dx, dy, ghosts)


#Initialize simulator
reload(FBL)
sim = FBL.FBL(gpu_ctx, \
              h0, eta0, u0, v0, \
              nx, ny, \
              dx, dy, dt, \
              g, f, r, \
              wind_stress=wind, \
              boundary_conditions=boundaryConditions)


#Calculate radius from center of bump for plotting
x_center = dx*nx*0.3
y_center = dy*ny*0.2
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))
   
fig = plt.figure()
eta1, u1, v1 = sim.download(interior_domain_only=True)
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, eta1, u1, v1)
T = 50
def animate(i):
    if (i>0):
        #t = sim.step(1.0)
        t = sim.step(10.0)
    else:
        t = 0.0
    eta1, u1, v1 = sim.download(interior_domain_only=True)

    plotter.plot(eta1, u1, v1);

    fig.suptitle("FBL Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%10 == 0):
        print "{:03.0f}".format(100.0*i / T) + " % => t=" + str(t)
        fig.savefig(imgdir + "/{:010.0f}_fbl.png".format(t))
            
anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim

In [ ]:
#Clean up old simulator if any:
if 'sim' in globals():
    sim.cleanUp()
    
#Centered in time, centered in space
nx = 100
ny = 200

dx = 200.0
dy = 200.0

dt = 1
g = 9.81
r = 0.0
A = 1
f = 0.01

#wind = WindStress.UniformAlongShoreWindStress(tau0=3.0, rho=1025, alpha=1.0/(100*dx)) 
wind =  WindStress.NoWindStress()

bcSettings = 1
ghosts = [1,1,1,1] # north, east, south, west
if (bcSettings == 1):
    boundaryConditions = Common.BoundaryConditions()
    # Wall boundary conditions
elif (bcSettings == 2):
    # periodic boundary conditions
    boundaryConditions = Common.BoundaryConditions(2,2,2,2)
elif bcSettings == 3:
    # periodic north-south
    boundaryConditions = Common.BoundaryConditions(2,1,2,1)
else:
    # periodic east-west
    boundaryConditions = Common.BoundaryConditions(1,2,1,2)


dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])


waterHeight = 60
h0 = np.ones(dataShape, dtype=np.float32, order='C') * waterHeight;
eta0 = np.zeros(dataShape, dtype=np.float32, order='C');
u0 = np.zeros((dataShape[0], dataShape[1]+1), dtype=np.float32, order='C');
v0 = np.zeros((dataShape[0]+1, dataShape[1]), dtype=np.float32, order='C');       
    

#Create bump in to lower left of domain for testing
x_center = dx*nx*0.3
y_center = dy*ny*0.2
makeCentralBump(eta0, 0.0, nx, ny, dx, dy, ghosts)
#makeUpperCornerBump(eta0, nx, ny, dx, dy, ghosts)
#addDualVortexStaggered(eta0, u0, v0, nx, ny, dx, dy, ghosts)
    

#Initialize simulator
reload(CTCS)
sim = CTCS.CTCS(gpu_ctx, \
                h0, eta0, u0, v0, \
                nx, ny, \
                dx, dy, dt, \
                g, f, r, A, \
                wind_stress=wind, \
                boundary_conditions=boundaryConditions )


#Calculate radius from center of bump for plotting
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))

ghosts = [-1,1,-1,1]

fig = plt.figure()
eta1, u1, v1 = sim.download(interior_domain_only=True)
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, eta1, u1, v1)

T = 50
def animate(i):
    if (i>0):
        t = sim.step(10.0)
    else:
        t = 0.0
    eta1, u1, v1 = sim.download(interior_domain_only=True)

    plotter.plot(eta1, u1, v1);
    fig.suptitle("CTCS Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%10 == 0):
        print "{:03.0f}".format(100.0*i / T) + " % => t=" + str(t)
        fig.savefig(imgdir + "/{:010.0f}_ctcs.png".format(t))

anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim

In [ ]:
#Clean up old simulator if any:
if 'sim' in globals():
    sim.cleanUp()
    
# Kurganov-Petrova 2007 paper
reload(KP07)
nx = 100
ny = 200

dx = 200.0
dy = 200.0

dt = 0.95/100
g = 9.81

f = 0.00
r = 0.0

# WIND
#wind = WindStress.UniformAlongShoreWindStress(tau0=3.0, rho=1025, alpha=1.0/(100*dx)) 
wind =  WindStress.NoWindStress()


bcSettings = 1
ghosts = np.array([2,2,2,2]) # north, east, south, west
if (bcSettings == 1):
    boundaryConditions = Common.BoundaryConditions()
elif (bcSettings == 2):
    boundaryConditions = Common.BoundaryConditions(2,2,2,2)
elif bcSettings == 3:
    # Periodic NS
    boundaryConditions = Common.BoundaryConditions(2,1,2,1)
else:
    # Periodic EW
    boundaryConditions = Common.BoundaryConditions(1,2,1,2)
    
dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])
waterHeight = 60
eta0 = np.zeros(dataShape, dtype=np.float32, order='C');
h0 = np.ones(dataShape, dtype=np.float32, order='C') * waterHeight;
u0 = np.zeros(dataShape, dtype=np.float32, order='C');
v0 = np.zeros(dataShape, dtype=np.float32, order='C');

Hi = np.ones((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C') * waterHeight;
#makeBathymetryCrater(Bi, nx+1, ny+1, dx, dy, ghosts)

#addCornerBump(h0, nx, ny, dx, dy, ghosts)
#addUpperCornerBump(h0, nx, ny, dx, dy, ghosts)
addCentralBump(eta0, nx, ny, dx, dy, ghosts)
#addLowerLeftBump(h0, nx, ny, dx, dy, ghosts)
#addDebugBump(h0, nx, ny, dx, dy, 0.2, 0.1, ghosts)
#addDiagonalDam(h0, nx, ny, dx, dy, ghosts, 0.05)
#addStraightDam(h0, nx, ny, dx, dy, ghosts, 0.1)


#Initialize simulator
reload(KP07)
reload(Common)
sim = KP07.KP07(gpu_ctx, \
                eta0, Hi, u0, v0, \
                nx, ny, \
                dx, dy, dt, \
                g, f, r, \
                wind_stress=wind, \
                boundary_conditions=boundaryConditions,
                use_rk2=True)


#Calculate radius from center of bump for plotting
x_center = dx*nx/2.0
y_center = dy*ny/2.0
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))


fig = plt.figure()
eta1, u1, v1 = sim.download(interior_domain_only=True)
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, eta1, u1, v1)

T = 50
def animate(i):
    if (i>0):
        t = sim.step(10.0)
    else:
        t = 0.0
    eta1, u1, v1 = sim.download(interior_domain_only=True)

    plotter.plot(eta1, u1, v1);
    fig.suptitle("KP07 Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%10 == 0):
        print "{:03.0f}".format(100*i / T) + " % => t=" + str(t) + "\tMax h: " + str(np.max(eta1))
        #fig.savefig(imgdir + "/{:010.0f}_kp07.png".format(t))
             
anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim

In [ ]:
#Clean up old simulator if any:
if 'sim' in globals():
    sim.cleanUp()
    
# Kurganov-Petrova 2007 paper without second order Runge Kutta
reload(KP07)
nx = 100
ny = 200

dx = 200.0
dy = 200.0

dt = 0.95/100
g = 9.81

f = 0.00
r = 0.0

# WIND
#wind = WindStress.UniformAlongShoreWindStress(tau0=3.0, rho=1025, alpha=1.0/(100*dx)) 
wind =  WindStress.NoWindStress()


bcSettings = 1
ghosts = np.array([2,2,2,2]) # north, east, south, west
if (bcSettings == 1):
    boundaryConditions = Common.BoundaryConditions()
elif (bcSettings == 2):
    boundaryConditions = Common.BoundaryConditions(2,2,2,2)
elif bcSettings == 3:
    # Periodic NS
    boundaryConditions = Common.BoundaryConditions(2,1,2,1)
else:
    # Periodic EW
    boundaryConditions = Common.BoundaryConditions(1,2,1,2)
    
dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])
waterHeight = 60
eta0 = np.zeros(dataShape, dtype=np.float32, order='C');
h0 = np.ones(dataShape, dtype=np.float32, order='C') * waterHeight;
u0 = np.zeros(dataShape, dtype=np.float32, order='C');
v0 = np.zeros(dataShape, dtype=np.float32, order='C');

Hi = np.ones((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C') * waterHeight;
#makeBathymetryCrater(Bi, nx+1, ny+1, dx, dy, ghosts)

#addCornerBump(h0, nx, ny, dx, dy, ghosts)
#addUpperCornerBump(h0, nx, ny, dx, dy, ghosts)
addCentralBump(eta0, nx, ny, dx, dy, ghosts)
#addLowerLeftBump(h0, nx, ny, dx, dy, ghosts)
#addDebugBump(h0, nx, ny, dx, dy, 0.2, 0.1, ghosts)
#addDiagonalDam(h0, nx, ny, dx, dy, ghosts, 0.05)
#addStraightDam(h0, nx, ny, dx, dy, ghosts, 0.1)


#Initialize simulator
reload(KP07)
reload(Common)
sim = KP07.KP07(gpu_ctx, \
                eta0, Hi, u0, v0, \
                nx, ny, \
                dx, dy, dt, \
                g, f, r, \
                wind_stress=wind, \
                boundary_conditions=boundaryConditions,
                use_rk2=False,
                write_netcdf=False)


#Calculate radius from center of bump for plotting
x_center = dx*nx/2.0
y_center = dy*ny/2.0
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))


fig = plt.figure()
eta1, u1, v1 = sim.download(interior_domain_only=True)
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, eta1, u1, v1)

T = 50
def animate(i):
    if (i>0):
        t = sim.step(10.0)
    else:
        t = 0.0
    eta1, u1, v1 = sim.download(interior_domain_only=True)

    plotter.plot(eta1, u1, v1)
    fig.suptitle("KP07 Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%10 == 0):
        print "{:03.0f}".format(100*i / T) + " % => t=" + str(t) + "\tMax eta: " + str(np.max(eta1))
        #fig.savefig(imgdir + "/{:010.0f}_kp07.png".format(t))
             
anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim

In [ ]:
#Clean up old simulator if any:
if 'sim' in globals():
    sim.cleanUp()
    
#Coriolis well balanced reconstruction scheme
nx = 100
ny = 200

dx = 200.0
dy = 200.0

dt = 0.95/10.0
g = 9.81

f = 0.00
r = 0.0

# WIND
#wind = WindStress.UniformAlongShoreWindStress(tau0=3.0, rho=1025, alpha=1.0/(100*dx)) 
#wind =  WindStress.NoWindStress()

# Test large-scale model errors emulated by perturbations of wind direction (for uniform wind forcing)
wind = WindStress.GenericUniformWindStress(wind_speed=3.0, wind_direction=90.0)



bcSettings = 1
ghosts = np.array([2,2,2,2]) # north, east, south, west
if (bcSettings == 1):
    boundaryConditions = Common.BoundaryConditions()
elif (bcSettings == 2):
    boundaryConditions = Common.BoundaryConditions(2,2,2,2)
elif bcSettings == 3:
    # Periodic NS
    boundaryConditions = Common.BoundaryConditions(2,1,2,1)
else:
    # Periodic EW
    boundaryConditions = Common.BoundaryConditions(1,2,1,2)
    
dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])


waterHeight = 60
eta0 = np.zeros(dataShape, dtype=np.float32, order='C');
h0 = np.ones(dataShape, dtype=np.float32, order='C') * waterHeight;
u0 = np.zeros(dataShape, dtype=np.float32, order='C');
v0 = np.zeros(dataShape, dtype=np.float32, order='C');

# Bathymetry:
Hi = np.ones((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C') * waterHeight;
#diagonalWallBathymetry(Bi, nx, ny, dx, dy, ghosts, 50)
#straightWallBathymetry(Bi, nx, ny, dx, dy, ghosts, 50)
#makeBathymetryCrater(Bi, nx+1, ny+1, dx, dy, ghosts)
#linearBathymetryX(Bi, nx, ny, dx, dy, ghosts, 5, 50)
#linearBathymetryY(Bi, nx, ny, dx, dy, ghosts, 5, 50)

#addCornerBump(h0, nx, ny, dx, dy, ghosts)
#addUpperCornerBump(h0, nx, ny, dx, dy, ghosts)
addCentralBump(eta0, nx, ny, dx, dy, ghosts)
#addStraightDam(h0, nx, ny, dx, dy, ghosts, 0.1)



#Initialize simulator
reload(CDKLM16)

sim = CDKLM16.CDKLM16(gpu_ctx, \
                eta0, u0, v0, \
                Hi, \
                nx, ny, \
                dx, dy, dt, \
                g, f, r, \
                max_wind_direction_perturbation=90.0, \
                wind_stress=wind, \
                boundary_conditions=boundaryConditions, \
                reportGeostrophicEquilibrium=False, \
                write_netcdf=False)

#Calculate radius from center of bump for plotting
x_center = dx*nx/2.0
y_center = dy*ny/2.0
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))

fig = plt.figure()
eta1, u1, v1 = sim.download(interior_domain_only=True)
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, eta1, u1, v1)

T = 50
def animate(i):
    if (i>0):
        t = sim.step(10.0)

    else:
        t = 0.0
    eta1, u1, v1 = sim.download(interior_domain_only=True)

    plotter.plot(eta1, u1, v1)
    
    fig.suptitle("CDKLM16 Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%10 == 0):
        print "{:03.0f}".format(100*i / T) + " % => t=" + str(t) + "\tMax eta: " + str(np.max(eta1)) + \
        "\tMax u: " + str(np.max(u1)) + \
        "\tMax v: " + str(np.max(v1))
        
        fig.savefig(imgdir + "/{:010.0f}_cdklm16.png".format(t))
             
anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim